# Hopfield Neural Network 5x5 RMSProp

In [ ]:
import tensorflow as tf
import numpy as np
from pylab import imshow, cm, show

## String versions of letters that will be learned by network 

In [ ]:
A = """
.XXX.
X...X
XXXXX
X...X
X...X
"""

Z = """
XXXXX
...X.
..X..
.X...
XXXXX
"""

E = """
XXXXX
X....
XXXX.
X....
XXXXX
"""

### Patternization and visualization functions

In [ ]:
def to_pattern(letter):
    return np.reshape(np.array([+1 if c=='X' else -1 for c in letter.replace('\n','')], dtype=np.float32), [1,25])

def display(pattern):
    imshow(pattern.reshape((5,5)),cmap=cm.binary, interpolation='nearest')
    show()

### Lets see the whole future memory set of network
Just to be sure, if the functions work correctly

In [ ]:
display(to_pattern(A))
display(to_pattern(Z))
display(to_pattern(E))

## Initialize weights

In [ ]:
t_patterns_prev = np.vstack([to_pattern(A), to_pattern(Z)])
t_patterns = np.vstack([t_patterns_prev, to_pattern(E)])
print('Patterns shape: '+str(t_patterns.shape))
print('Patterns:\n'+str(t_patterns))

def init(patterns):
    r,c = patterns.shape
    w = np.round(np.random.rand(c,c), decimals=3)
    w[np.diag_indices(c)] = 0
    return w

W = tf.Variable(init(t_patterns), dtype=tf.float32, name='w')
W_ = init(t_patterns)

print('Shape of weight matrix: '+str(W_.shape))
print('Weights:\n'+str(W_))

## Set up the model
### We will use not only one parasitic energy point of a memory, but 5 at once and try to maximize them, while decreasing the energy of target memories
We will use sigmoid function and then round results, because it is a binary decision for a neural network. Of couse it seems to be more legit to use the 1/-1 versions of final neuron activations as we used in making patterns from letters, but we cannot round tanh function results properly without getting zeros and apply special functions to every scalar every time without hardcoding and using boolean logic. It does not matter, if we will suddenly change the whole perspective in this case.

In [ ]:
X = tf.placeholder(tf.float32, [1, 25], name='X')

with tf.name_scope('Y_1'):
    Y_prev_1 = tf.matmul(X,W)
    Y_1 = tf.round(tf.nn.sigmoid(Y_prev_1))

with tf.name_scope('Y_2'):
    Y_prev_2 = tf.matmul(Y_prev_1, W)
    Y_2 = tf.round(tf.nn.sigmoid(Y_prev_2))

with tf.name_scope('Y_3'):
    Y_prev_3 = tf.matmul(Y_prev_2, W)
    Y_3 = tf.round(tf.nn.sigmoid(Y_prev_3))

with tf.name_scope('Y_4'):
    Y_prev_4 = tf.matmul(Y_prev_3, W)
    Y_4 = tf.round(tf.nn.sigmoid(Y_prev_4))

with tf.name_scope('Y_5'):
    Y_prev_5 = tf.matmul(Y_prev_4, W)
    Y_5 = tf.round(tf.nn.sigmoid(Y_prev_5))

with tf.name_scope('t_energy'):
    target_energy = tf.reshape(-0.5*tf.matmul(tf.matmul(X, W), X, transpose_b=True), [])
    
with tf.name_scope('p_energy'):
    with tf.name_scope('p_energy_1'):
        parasitic_energy_1 = tf.reshape(-0.5*tf.matmul(tf.matmul(Y_1,W),Y_1, transpose_b=True), [])
    with tf.name_scope('p_energy_2'):
        parasitic_energy_2 = tf.reshape(-0.5*tf.matmul(tf.matmul(Y_2,W),Y_2, transpose_b=True), [])
    with tf.name_scope('p_energy_3'):
        parasitic_energy_3 = tf.reshape(-0.5*tf.matmul(tf.matmul(Y_3,W),Y_3, transpose_b=True), [])
    with tf.name_scope('p_energy_4'):
        parasitic_energy_4 = tf.reshape(-0.5*tf.matmul(tf.matmul(Y_4,W),Y_4, transpose_b=True), [])
    with tf.name_scope('p_energy_5'):
        parasitic_energy_5 = tf.reshape(-0.5*tf.matmul(tf.matmul(Y_5,W),Y_5, transpose_b=True), [])
    with tf.name_scope('p_m_energy'):
        parasitic_mean_energy = tf.reshape(tf.divide(tf.add_n([parasitic_energy_1, parasitic_energy_2, parasitic_energy_3, parasitic_energy_4, parasitic_energy_5]), 5.0), [])

tf.summary.scalar('t_energy_sc', target_energy)
tf.summary.scalar('p_energy_1_sc', parasitic_energy_1)
tf.summary.scalar('p_energy_2_sc', parasitic_energy_2)
tf.summary.scalar('p_energy_3_sc', parasitic_energy_3)
tf.summary.scalar('p_energy_4_sc', parasitic_energy_4)
tf.summary.scalar('p_energy_5_sc', parasitic_energy_5)
tf.summary.scalar('p_m_energy_sc', parasitic_mean_energy)

saver = tf.train.Saver()

optimizer = tf.train.RMSPropOptimizer(learning_rate=0.01, decay=0.001).minimize(target_energy-parasitic_mean_energy)

## Start the Session
### There is no point to run it for a very long time because you will almost never delete all parasitic memories by decreasing energy
So if you have a much more difficult task it is better to teach network to escape from parasites, not decrease energy level of part of them, but it surely can help a little bit. The main idea is that you should use Boltzmann Machines (basically Hopfield Nets with probability), if deterministic version of Hopfield Net has failed, not overfit the original Hopfield Net.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./graph', sess.graph)
    for i in xrange(10000):
        if i == 0:
            num = 1
        _, t_e_1, p_e_1, summary_1 = sess.run([optimizer, target_energy, parasitic_mean_energy, merged], feed_dict={X: to_pattern(A)})
        print('Step '+str(num)+':')
        print('Target energy of A: '+str(t_e_1))
        print('   Parasitic energy of A: '+str(p_e_1))
        writer.add_summary(summary_1, num)
        num+=1
        _, t_e_2, p_e_2, summary_2 = sess.run([optimizer, target_energy, parasitic_mean_energy, merged], feed_dict={X: to_pattern(Z)})
        print('Step '+str(num)+':')
        print('Target energy of Z: '+str(t_e_2))
        print('   Parasitic energy of Z: '+str(p_e_2))
        writer.add_summary(summary_2, num)
        num+=1
        _, t_e_3, p_e_3, summary_3 = sess.run([optimizer, target_energy, parasitic_mean_energy, merged], feed_dict={X: to_pattern(E)})
        print('Step '+str(num)+':')
        print('Target energy of E: '+str(t_e_3))
        print('   Parasitic energy of E: '+str(p_e_3))
        writer.add_summary(summary_3, num)
        num+=1
    weights = W.eval()
    print('Weights of trained model:\n'+str(weights))
    save_path = saver.save(sess, './model/hopfield')
    print('Model has been successfully trained and saved')

Thats it!